In [1]:
%pwd

'c:\\Users\\asif8\\OneDrive\\Documents\\practice-work\\genai\\medical-chatbot-genai\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\asif8\\OneDrive\\Documents\\practice-work\\genai\\medical-chatbot-genai'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents = loader.load()
    
    return documents

In [6]:
# Load the PDF file
extracted_data = load_pdf_file(data="Data/")

In [9]:
#extracted_data

In [10]:
# Split the data into chunks
def split_data(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    # Split the data into chunks
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks


In [11]:
text_chunks = split_data(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  39994


## Embedding the data into the vector database

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [21]:
# Download the Embeddings from HuggingFace
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [23]:
embeddings = download_hugging_face_embeddings()

In [26]:
query_result = embeddings.embed_query("What is the name of the patient?")
print("Length: ",len(query_result))

Length:  384


## Create Pinecone index

In [28]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [35]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    index_name,
    dimension=384, 
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-mtvkost.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

## Save the embeddings in the Pinecone Vector Store

In [38]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks, 
    index_name=index_name,
    embeddings=embeddings
)

TypeError: VectorStore.from_documents() missing 1 required positional argument: 'embedding'